<a href="https://colab.research.google.com/github/denisparra/diplomadoInfoVis/blob/master/Diplomado_Alumno_2019_Reducci%C3%B3n_de_dimensionalidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Diplomado _Big Data_ - Reducción de dimensionalidad

**Profesores:** Nebil Kawas y Denis Parra.

**Ayudantes**: Juan Espinosa, Germán Contreras y Hernán Valdivieso.

Guía creada por Ivania Donoso y Hernán Valdivieso

-----


---
El siguiente _jupyter notebook_ tiene como fin ser una breve guía de creación de gráficos con interacciones utilizando la librería altair y de utilización de reducción de dimensionalidad para algunos problemas interesantes. Vamos a utilizar los datos de los resultados por comuna de las elecciones presidenciales de Chile desde el año 1989.

Para facilitar la instalación de esta librería, este archivo debe ser subido a [Google Colab](https://colab.research.google.com/) para poder ejecutarlo en los servidores de Google, los cuales  disponen de todos los recursos necesarios para ejecutar los códigos que implementarás.

Este  _jupyter notebook_ se divide en 2 partes:

- Utilización de interacciones de Altair
- Reducción de dimensionalidad.

# Integrantes

Este práctico se puede realizar en parejas.

- Nombre 1
- Nombre 2

Recuerde subir el PDF y .ipynb de este práctico. **No suba un .zip con todo**

# Indice  


>[Diplomado Big Data - Reducción de dimensionalidad](#scrollTo=xCXryKWzePSi)

>[Indice](#scrollTo=xCXryKWzePSi)

>>[Descargar información](#scrollTo=9AdImEEudzLv)

>>[Importar librerías del práctico](#scrollTo=p-zksDqUd1kx)

>[Introducción sobre los datos](#scrollTo=46Aeme6b4FyH)

>[Parte 1: Usar las columnas de resultados](#scrollTo=otcBVLQl4FyV)

>>[Scatter matrix](#scrollTo=otcBVLQl4FyV)

>>[Actividad 1](#scrollTo=dCaXxLRQ4FyY)

>>[Más interacciones](#scrollTo=30Mn2VQn4Fyd)

>>[Actividad 2](#scrollTo=KtYEgI8X4Fyg)

>[Parte 2: Usar reducción de dimensionalidad](#scrollTo=1nJjLAEM4Fyk)

>>[Reducción de dimensión en imágenes](#scrollTo=JO4mQXHo4Fyr)

>>>[PCA](#scrollTo=1d42BMv5QOH_)

>>>>[Observación](#scrollTo=YVSerMbLRA2R)

>>>[TSNE](#scrollTo=YVSerMbLRA2R)

>>>>[Observación](#scrollTo=7fxY5VuocjDI)

>>>[UMAP](#scrollTo=7fxY5VuocjDI)

>>>>[Observación](#scrollTo=cnvK7mQBc441)

>>[Actividad 3](#scrollTo=FCIvPNcREs6s)

>>>[Descargar dataset](#scrollTo=FCIvPNcREs6s)

>>>[Cargar información](#scrollTo=II-Hwi0SKi9-)

>>>[Funciones](#scrollTo=cqClLcTHGped)

>>>[Documentación de funciones](#scrollTo=eqaNy1ISFN7x)

>>>[Generar gráficos](#scrollTo=WAdwTZaCFrB_)

>>>[¿Con ganas de ver más?](#scrollTo=JwM1UDfmOvJa)

>>[Actividad 4](#scrollTo=ADLHEZ_mRfXT)

>>>[Documentación de funciones (la misma que de la actividad 03)](#scrollTo=ADLHEZ_mRfXT)

>>>[Completar el siguiente código y luego responder la pregunta](#scrollTo=xNPO38qxjsXr)

>>>[Respuesta](#scrollTo=Qmyfy5PXZTkj)



## Descargar información

In [0]:
from IPython.display import clear_output


!wget -N https://www.dropbox.com/s/l76bz6z65pm1gcw/resultado_presidente_comunas.csv
!wget -N https://www.dropbox.com/s/3gsq3wng910cvwr/diccionario_hechos
  
clear_output() # Limpiar ventana para que no se vea despues.
!ls

## Importar librerías del práctico

In [0]:
import pandas as pd
import json
import pprint
from vega_datasets import data
import altair as alt
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.manifold import TSNE, MDS
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import umap
from sklearn import datasets
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


# Introducción sobre los datos

En este laboratorio, vamos a trabajar con los resultados de las elecciones presidenciales de Chile desde 1989. Los resultados están guardados en una tabla donde cada fila es una comuna. Hay dos tipos de columnas: las que muestran el porcentaje que obtuvo un candidato en una elección y las columnas de valor binario que muestran la presencia o ausencia de un hecho en una comuna. Además, existe un diccionario que explica qué significan los hechos.

Los datos que tienen los resultados por comuna están en un archivo csv

In [0]:
df = pd.read_csv('resultado_presidente_comunas.csv')  # Cargamos los datos
df.head()

Las columnas de candidatos son las siguientes:

In [0]:
columnas_candidatos = ['a-alessandri_1993_1', 'a-frei_1999_1', 'a-guillier_2017_1',
       'a-guillier_2017_2', 'a-navarro_2017_1', 'a-sfeir_2013_1',
       'b-sanchez_2017_1', 'c-goic_2017_1', 'c-reitze_1993_1',
       'e-artes_2017_1', 'e-frei_1993_1', 'e-frei_2009_1', 'e-frei_2009_2',
       'e-matthei_2013_1', 'e-matthei_2013_2', 'e-pizarro_1993_1',
       'f-errazuriz_1989_1', 'f-parisi_2013_1', 'g-marin_1999_1',
       'h-buchi_1989_1', 'j-a-kast_2017_1', 'j-arrate_2009_1',
       'j-lavin_1999_1', 'j-lavin_1999_2', 'j-lavin_2005_1', 'j-pinera_1993_1',
       'm-bachelet_2005_1', 'm-bachelet_2005_2', 'm-bachelet_2013_1',
       'm-bachelet_2013_2', 'm-claude_2013_1', 'm-enriquez-o_2009_1',
       'm-enriquez-o_2017_1', 'm-enriquez-ominami_2013_1', 'm-maxneef_1993_1',
       'p-aylwin_1989_1', 'r-israel_2013_1', 'r-lagos_1999_1',
       'r-lagos_1999_2', 'r-miranda_2013_1', 's-larrain_1999_1',
       's-pinera_2005_1', 's-pinera_2005_2', 's-pinera_2009_1',
       's-pinera_2009_2', 's-pinera_2017_1', 's-pinera_2017_2',
       't-hirsch_1999_1', 't-hirsch_2005_1', 't-jocelyn-holt_2013_1']

Cada columna de candidato está descrita como: `{iniciales_nombre}-{apellido_paterno}_{año_eleccion}_{primera_o_segunda_vuelta}`.

Las columnas de hechos son 16. Cada una de estas describe una posible característica de la comuna de acuerdo a las elecciones de 2017. Cada una tiene un nombre (`name`), un nombre descriptivo (`label`) y una descripción (`description`). A continuación pueden ver los diferentes hechos:

In [0]:
pp = pprint.PrettyPrinter(indent=2)

with open('diccionario_hechos') as f:
    descripcion_hechos = json.load(f)
    
print(json.dumps(descripcion_hechos, indent=4, ensure_ascii=False))
# json.dumps es solo para que se vea bonito el print

Para seleccionar y ver las característica de un eje solo debe seleccionarlo de la siguiente forma:

In [0]:
print(descripcion_hechos['wins-pinera'])
print("Label: ", descripcion_hechos['wins-pinera']['label'])

Por ejemplo, si vemos la primera y segunda columna,,  en la comuna de **Algarrobo** tenemos que **37.29 %** de los habitantes votó por **Arturo Alessandri**. También, si vemos las últimas columnas, tenemos que la comuna tiene presente el hecho `wins-pinera` que mostramos arriba.

In [0]:
pd.options.display.max_columns = None
df.head(1)

En la actividad de hoy partiremos usando algunos gráficos con interacción con las columnas tal y como vienen en el dataset, y luego utilizaremos reducción de dimensionalidad para obtener más información de los datos. Luego veremos una aplicación diferente de reducción de dimensionalidad para finalmente hacer una actividad final donde apliquen estas técnicas y visualicen un _dataset_ distinto.

# Parte 1: Usar las columnas de resultados
## Scatter matrix
Un _scatter matrix_ es varios _scatter plot_ en donde cada gráfico es una combinación de dos variables del dataset. Este tipo de visualización permite obtener información rápidamente sobre la correlación entre dos variables.

In [0]:
cars = data.cars()
cars.head(3)

In [0]:
import altair as alt

# Un brush nos permite seleccionar varios elementos de la visualización al mismo tiempo
brush = alt.selection(type='interval')

# Para hacer este gráfico usaremos la propiedad repeat. 
alt.Chart(cars).mark_circle().encode(
    # Lo que esté en la variable column estará en el eje X
    alt.X(alt.repeat("column"), type='quantitative'),
    
    # Lo que esté en la variable row estará en el eje y
    alt.Y(alt.repeat("row"), type='quantitative'),
    
    # El color de cada elemento depende de una condición.
    # Si está seleccionado con un brush será del color que corresponda a 'Origin:N',
    # en otro caso será de color gris.
    # Legend nos permite cambiar la leyenda.
    color=alt.condition(brush, 'Origin:N', alt.ColorValue('gray'), legend=alt.Legend(title="Country of Origin")),
).add_selection(
    # Agregamos el objeto brush que declaramos arriba
    brush 
).properties( 
    # Modificamos el tamaño del gráfico para que se ajuste a la pantalla. 
    width=150, 
    height=150,
).repeat( 
    # Estas son las columnas que se van a repetir. 
    row=['Horsepower', 'Acceleration', 'Miles_per_Gallon'],
    column=['Miles_per_Gallon', 'Acceleration', 'Horsepower']
)

## Actividad 1
Realice un `scatter matrix` que muestre la votación de la primera vuelta del año 2005. La matriz debe mostrar las combicaciones de todos los candidatos del año 2005. El color de cada columa debe indicar un hecho, puede elegir cualquiera de los 16 hechos. El título de la leyenda de los colores debe mostrar el nombre en nombre descriptivo del hecho.

Ayuda: esta sentencia selecciona todos los columnas que corresponden a candidatos de la año 2005 en primera vuelta

In [0]:
candidates = [c for c in columnas_candidatos if '2005_1' in  c]

In [0]:
# Cambiar los 4 "COMPLETAR" por lo necesario

alt.Chart(df).mark_circle().encode(
    alt.X(alt.repeat("column"), type='quantitative',  axis=alt.Axis(format='%')),
    alt.Y(alt.repeat("row"), type='quantitative', axis=alt.Axis(format='%')),
    color=alt.condition(brush, "COMPLETAR", alt.ColorValue('gray'), legend=alt.Legend(title="COMPLETAR")),
).add_selection(
    brush  
).properties(
    width=150,
    height=150
).repeat(
    row= "COMPLETAR",
    column="COMPLETAR"
)

## Más interacciones

En este ejemplo usamos 3 gráficos que están coordinados. Estamos haciendo un _select_,  _juxtapose_ y _embed_. 

In [0]:
cars = data.cars()

brush = alt.selection(type='interval')

# Estas dos variables nos permiten definir si queremos los labels de los valores de los ejes y sus títulos. 
tick_axis = alt.Axis(labels=False, domain=False, ticks=False)
tick_axis_notitle = alt.Axis(labels=False, domain=False, ticks=False, title='')

points = alt.Chart(cars).mark_point().encode(
    x=alt.X('Miles_per_Gallon', axis=alt.Axis(title='')),
    y=alt.Y('Horsepower', axis=alt.Axis(title='')),
    color=alt.condition(brush, 'Origin', alt.value('grey')),
    tooltip=['Name']  # Aquí estamos usando embed para mostrar más información de los datos
).add_selection(
    brush
)

# Para el gráfico que va en el eje X no queremos mostrar el título en el eje y
x_ticks = alt.Chart(cars).mark_tick().encode(
    alt.X('Miles_per_Gallon', axis=tick_axis),  # Queremos que el título del eje X sí aparezca.
    alt.Y('Origin', axis=tick_axis_notitle),  # Ponemos la variable que elimina los títulos.
    color=alt.condition(brush, 'Origin', alt.value('lightgrey'))
).add_selection(
    brush
)

y_ticks = alt.Chart(cars).mark_tick().encode(
    alt.X('Origin', axis=tick_axis_notitle),
    alt.Y('Horsepower', axis=tick_axis),
    color=alt.condition(brush, 'Origin', alt.value('lightgrey'))
).add_selection(
    brush
)

# Se disponen los gráficos en columnas y filas. Las barras verticales | definen las columnas y los & definen filas.

y_ticks | (points & x_ticks)  


## Actividad 2

Realice un gráfico como el de arriba para dos candidatos de una misma elección. El tootlip debe mostrar el nombre de la comuna y el color debe mostrar un hecho a elección.

In [0]:
# Cambiar los 6 "COMPLETAR" por lo necesario

brush = alt.selection(type='interval')

# Indicar un hecho
hecho = "COMPLETAR"

# Estas dos variables nos permiten definir si queremos los labels de los valores de los ejes y sus títulos. 
tick_axis = alt.Axis(labels=False, domain=False, ticks=False)
tick_axis_notitle = alt.Axis(labels=False, domain=False, ticks=False, title='')

points = alt.Chart(df).mark_point().encode(
    x=alt.X("COMPLETAR", axis=alt.Axis(title='')),
    y=alt.Y("COMPLETAR", axis=alt.Axis(title='')),
    color=alt.condition(brush, hecho + ":N", alt.value('grey'), legend=alt.Legend(title=descripcion_hechos[hecho]['label'])),
    tooltip="COMPLETAR"
).add_selection(
    brush
)

# Para el gráfico que va en el eje X no queremos mostrar el título en el eje y
x_ticks = alt.Chart(df).mark_tick().encode(
    alt.X("COMPLETAR", axis=tick_axis),  # Queremos que el título del eje X sí aparezca.
    alt.Y(hecho + ":N", axis=tick_axis_notitle),  # Ponemos la variable que elimina los títulos.
    color=alt.condition(brush, hecho + ":N", alt.value('lightgrey'))
).add_selection(
    brush
)

y_ticks = alt.Chart(df).mark_tick().encode(
    alt.X(hecho + ":N", axis=tick_axis_notitle),
    alt.Y("COMPLETAR", axis=tick_axis),
    color=alt.condition(brush, hecho + ":N", alt.value('lightgrey'))
).add_selection(
    brush
)

y_ticks | (points & x_ticks)  

# Parte 2: Usar reducción de dimensionalidad

Reducción de dimensionalidad es transformar los datos a una nueva representación con menos dimensiones. Por ejemplo, tener 10 columnas iniciales y luego tener 2 columnas que representan más o menos la misma información.

Existen muchos tipos de formas de reducir la dimensionalidad. Unas son supervisadas, como _LDA_, pero la mayoría son no supervisadas, es decir no necesitamos clases. En esta actividad veremos 5: _PCA_, _LDA_, _LSA_, _t-SNE_, _MDS_.

In [0]:
def principal_components_analysis(data, componentes, **kwargs):
    """
    Explicación del método en español: https://es.wikipedia.org/wiki/An%C3%A1lisis_de_componentes_principales
    """
    dim_red = PCA(n_components=componentes).fit(data)
    data_new_dimensions = dim_red.transform(data)
    return data_new_dimensions


def linear_discriminant_analysis(data, componentes, classes):
    """
    Esta es otra técnica que es posible utilizar, pero no fue vista en clases.
    Explicación del método en español: https://es.wikipedia.org/wiki/An%C3%A1lisis_discriminante_lineal
    
    Este método entrega como máximo el número de clases -1 componentes. Es decir, si tenemos dos clases, entrega una componente.
    """
    dim_red = LinearDiscriminantAnalysis(solver='svd', n_components=componentes+1).fit(data, classes)
    data_new_dimensions = dim_red.transform(data)
    return data_new_dimensions

def tsne(data, componentes, **kwargs):
    """
    Expicación del método en inglés: https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding
    
    Este método tiene una componente aleatoria, por lo que no siempre entrega la misma distribución para un set de puntos.
    """
    data_new_dimensions = TSNE(n_components=componentes).fit_transform(data)
    return data_new_dimensions


def multidimensional_scalling(data, componentes, **kwargs):
    """
    Esta es otra técnica que es posible utilizar, pero no fue vista en clases.
    Explicación del método en español: https://es.wikipedia.org/wiki/Escalamiento_multidimensional
    Este método tiene una componente aleatoria, por lo que no siempre entrega la misma distribución para un set de puntos.
    """
    data_new_dimensions = MDS(n_components=componentes).fit_transform(data)
    return data_new_dimensions
  
def uniform_manifold_approximation_and_projection(data, componentes, **kwargs):
    """
    Explicación del método en ingles: https://umap-learn.readthedocs.io/en/latest/
    Paper: https://arxiv.org/pdf/1802.03426.pdf
    Este método tiene una componente aleatoria, por lo que no siempre entrega la misma distribución para un set de puntos.
    """
    data_new_dimensions = umap.UMAP().fit_transform(data)
    return data_new_dimensions

# Este diccionario nos permitirá seleccionar un método solo con sus iniciales.
TRANSFORMATIONS = {
    'pca': principal_components_analysis,
    'lda': linear_discriminant_analysis, 
    'tsne': tsne,
    'mds': multidimensional_scalling,
    'umap': uniform_manifold_approximation_and_projection
}


def reduce_dimensions(transformation, data_original, columnas, clases=None, componentes=2):
    """
    Este método es que el que deben llamar para obtener los datos. 
    Les agregará las componentes que encuentre al dataframe que ya estaban utilizando.
    
    :param transformation: transformación que quieren realizar: pca, lsa, lda, tsne, mds
    :param data_original: el dataframe de los datos que se van a transformar
    :param columnas: columnas sobre las que se aplicará la transformación\
    :param clases: necesaria para el método lda. Las clases que se utilizarán para separar los datos
    :param componentes: número de componentes que se obtendrán. 
    :return el dataframe con las componentes
    """
    if transformation == 'lda' and clases is None:
        raise ValueError('Linear Discriminant Analysis necesita las clases')
    
    try:
        transform_method = TRANSFORMATIONS[transformation]
    except KeyError as e: 
        raise ValueError(
            'La transformación {0} no está disponible'.format(transformation))
    
    data = data_original.copy()
    converted_data = transform_method(data[columnas], componentes, classes=clases)
    for component in range(converted_data.shape[1]):        
        data['c{}'.format(component)] = converted_data[:, component]
    return data

En este ejemplo veremos cómo podemos armar una visualización con usando las reducciones de dimensionalidad.

In [0]:
# Vamos a usar como brush uno que funciona en mouseover. 
# El parámetro fields indica que en cada gráfico seleccionará la marca que corresponda a la misma comuna.

brush = alt.selection_multi(on='mouseover', fields=['comuna'])


# Esta función creará algunos de los gráficos que vamos a usar
def make_chart(columna_color, tipo, data, titulo):
    """
    :param columna_color: el nombre de la columna que entrega el color de los círculos,
    :param tipo: tipo de la columna del color: N, Q u O
    :param data: dataframe con los datos
    :return un Chart de Altair
    """
    return  alt.Chart(data).mark_circle(size=100).encode(
        x='c0',
        y='c1',
        color=alt.condition(brush, columna_color + ':' + tipo, alt.value('lightgrey'),
                            legend=alt.Legend(title=descripcion_hechos[columna_color]['label'])),
        tooltip=['comuna'],
        # Esta configuración de opacidad nos permite ver mejor el elemento que estamos seleccionando.
        opacity= alt.condition(brush, alt.OpacityValue(1), alt.OpacityValue(0.1))
    ).add_selection(
        brush  # Este el brush es el mismo que definimos arriba.
    ).properties(
        width=250,
        height=250,
        title=titulo
    )


# La columna del color que vamos a usar. 
columna_color = 'wins-derecha'
tipo = 'N'

# Vamos a usar los candidatos del año 1993
candidatos_1993 = [c for c in columnas_candidatos if '1993' in c]

# Creamos los datos para cada uno de los métodos de reducción que tenemos.
pca_data = reduce_dimensions('pca', df, candidatos_1993)
tsne_data = reduce_dimensions('tsne', df,  candidatos_1993)
mds_data = reduce_dimensions('mds', df,  candidatos_1993)
umap_data = reduce_dimensions('umap', df,  candidatos_1993)
lda_data = reduce_dimensions('lda', df,  candidatos_1993, clases=df[columna_color])


# Se crean los gráficos para cada uno de los datos.
pca_chart = make_chart(columna_color, tipo, pca_data, 'PCA')
tsne_chart = make_chart(columna_color, tipo, tsne_data, 't-SNE')
mds_chart = make_chart(columna_color, tipo, mds_data, 'MDS')
umap_chart = make_chart(columna_color, tipo, umap_data, 'UMAP')


# En el caso de LDA no podemos hacer scatterplot porque solo tenemos dos clases, por lo tanto solo una componente. 
# Vamos a hacer el mismo gráfico con ticks que hicimos arriba.

# Para el gráfico que va en el eje X no queremos mostrar el título en el eje y
tick_axis = alt.Axis(labels=False, domain=False, ticks=False)
tick_axis_notitle = alt.Axis(labels=False, domain=False, ticks=False, title='')

lda_chart = alt.Chart(lda_data).mark_tick(size=30).encode(
        alt.X('c0', axis=tick_axis),  # Queremos que el título del eje X sí aparezca.
        alt.Y(columna_color + ':' + tipo, axis=tick_axis_notitle),  # Ponemos la variable que elimina los títulos.
        color=alt.condition(brush, columna_color + ':' + tipo, alt.value('lightgrey'), legend=alt.Legend(title=descripcion_hechos[columna_color]['label'])),
        tooltip=['comuna'],
        # Aplicamos la misma configuración de opacidad que arriba
        opacity= alt.condition(brush, alt.OpacityValue(1), alt.OpacityValue(0.1))
    ).add_selection(
        brush  # Tenemos que usar el mismo brush que para los otros gráficos.
    ).properties(
    # Lo vamos a hacer más ancho para que quede más al centro.
    width=500, height=90, title='LDA')

# Los organizamos y mostramos.
(((pca_chart & umap_chart)|( mds_chart & tsne_chart)) & lda_chart).configure(tick=alt.TickConfig(thickness=3))  # Este configure nos permite hacer que las líneas del gráfico de LDA se configuren

## Reducción de dimensión en imágenes

A continuación vamos a tomar diferentes diferentes imágenes de letras escritas a mano, estas son imágenes de 8 x 8 pixeles, por lo tanto podemos decir que para cada imagen hay información de 64 pixeles, es decir, 64 columnas.

In [0]:
N = 100 # Cantidad de imagenes a trabajar

dataset = datasets.load_digits()
data_digits = np.array([x.flatten() for x in dataset.images[:N]])

fig, ax = plt.subplots(4, 8, subplot_kw=dict(xticks=[], yticks=[]))
for i, axi in enumerate(ax.flat):
    axi.imshow(dataset.images[i], cmap='gray')

In [0]:
from matplotlib import offsetbox

def plot_components(data, model, images=None, ax=None,
                    thumb_frac=0.01):
    ax = ax or plt.gca()
    
    proj = model.fit_transform(data)
    ax.plot(proj[:, 0], proj[:, 1], '.k')
    
    if images is not None:
        min_dist_2 = (thumb_frac * max(proj.max(0) - proj.min(0))) ** 2
        shown_images = np.array([2 * proj.max(0)])
        for i in range(data.shape[0]):
            dist = np.sum((proj[i] - shown_images) ** 2, 1)
            if np.min(dist) < min_dist_2:
                # don't show points that are too close
                continue
            shown_images = np.vstack([shown_images, proj[i]])
            imagebox = offsetbox.AnnotationBbox(offsetbox.OffsetImage(images[i],3,cmap="gray"),proj[i])
            ax.add_artist(imagebox)

### PCA

Vamos a probar reducir esas 64 columnas por imagen en 2 columnas, luego, utilizar esas 2 columnas como coordenadas (x,y) para posicionar la imagen y ver como se organizan estas.

In [0]:
fig, ax = plt.subplots(figsize=(18, 10))
plot_components(data_digits, model=PCA(n_components=2), images=dataset.images[:N])

#### Observación
Podemos ver que algunos números se agrupan entre si, estos es porque la información de sus 64 columnas es similar y cuando se aplica la reducción de dimensión, las 2 columnas que quedan contienen casi la misma información y por ello, van a tender a estar juntas las imágenes similares. Ahora utilicemos otra técnica de reducción.

### TSNE

https://distill.pub/2016/misread-tsne/

In [0]:
perplexity = 50

fig, ax = plt.subplots(figsize=(18, 10))
plot_components(data_digits, model=TSNE(n_components=2, perplexity=perplexity), images=dataset.images[:N])

#### Observación
Podemos apreciar que ahora si las imágenes están mucho mejor agrupadas, esto demuestra que para este caso particular, utilizar TSNE permite segmentar mejor que LDA, pero que tal probar otro método más.

### UMAP

In [0]:
n_neighbors = 10 #Valor de 2 a 100

fig, ax = plt.subplots(figsize=(18, 10))
plot_components(data_digits, model=umap.UMAP(n_neighbors=n_neighbors), images=dataset.images[:N])

#### Observación

Podemos apreciar que bajo UMAP,  los números quedaron mucho más agrupados entre si y separados de otros, lo cual permite mostrar que esta reducción de dimensión logra bajar de 64 columnas a 2 y mantiene bastante información de los datos para poder seguir distinguindo una clase de otra.

## Actividad 3

A continuación se cargará un dataset con diferentes imágenes de obras de arte, cada imagen tiene asociado un vector de 2048 datos, es decir, 2048 columnas. Aplique 2 técnicas de reducción de dimensionalidad a dicho dataset y visualice las imágenes en una grilla como el ejemplo anterior. Para esto se entregará una serie de funciones para facilitar este proceso.

### Descargar dataset



In [0]:
from IPython.display import clear_output


!wget -N https://www.dropbox.com/s/xh0twkvvbh5y7ug/surdoc.zip
!unzip -o surdoc
  
clear_output() # Limpiar ventana para que no se vea despues.
!ls

### Cargar información

Esto cargará 100 imagenes de las aproximadamente 400 que hay, cada vez que ejecutes la siguiente casilla, se seleccionarán 100 de forma aleatoria

In [0]:
import os
import random
import cv2


images = []
dataset = []
images_path = random.sample(os.listdir("images"), 100)
for image in images_path:
  if image.endswith(".jpg"):
    image_file = cv2.imread(os.path.join("images", image))
    resized = cv2.resize(image_file, (20, 30), interpolation = cv2.INTER_AREA)
    images.append(resized)

    npy = image[:-3] + "npy"
    dataset.append(np.load(os.path.join("features", npy)))
  
images_surdoc = np.array(images)
dataset_surdoc = np.array(dataset)

### Funciones

A continuación se presentan todas las funciones a utilizar, en el siguiente bloque se documenta cada una

In [0]:
def reducir_con_pca(data):
    """
    Principal Components Analysis (pca)
    Explicación del método en español: https://es.wikipedia.org/wiki/An%C3%A1lisis_de_componentes_principales
    """
    dim_red = PCA(n_components=2).fit(data)
    data_new_dimensions = dim_red.transform(data)
    return pd.DataFrame(data_new_dimensions, columns=["X", "Y"])


def reducir_con_tsne(data, perplexity):
    """
    Expicación del método en inglés: https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding
    
    Este método tiene una componente aleatoria, por lo que no siempre entrega la misma distribución para un set de puntos.
    """
    data_new_dimensions = TSNE(n_components=2, perplexity=perplexity).fit_transform(data)
    return pd.DataFrame(data_new_dimensions, columns=["X", "Y"])


def reducir_con_mds(data, max_iter):
    """
    Esta es otra técnica que es posible utilizar, pero no fue vista en clases.
    Multidimensional Scalling (mds)
    Explicación del método en español: https://es.wikipedia.org/wiki/Escalamiento_multidimensional
    Este método tiene una componente aleatoria, por lo que no siempre entrega la misma distribución para un set de puntos.
    """
    data_new_dimensions = MDS(n_components=2, max_iter=max_iter).fit_transform(data)
    return pd.DataFrame(data_new_dimensions, columns=["X", "Y"])

def reducir_con_umap(data, n_neighbors):
    """
    Multidimensional Scalling (mds)
    Explicación del método en español: https://es.wikipedia.org/wiki/Escalamiento_multidimensional
    Este método tiene una componente aleatoria, por lo que no siempre entrega la misma distribución para un set de puntos.
    """
    data_new_dimensions = umap.UMAP(n_neighbors=n_neighbors).fit_transform(data)
    return pd.DataFrame(data_new_dimensions, columns=["X", "Y"])

def visualizar(dataset_reducido, metodo):
  """
  Método utilizado para visualizar las obras de arte
  """
  print(metodo)
  fig, ax = plt.subplots(figsize=(25, 10)) # AQUÍ PUEDEN CAMBIAR EL TAMAÑO DEL GRAFICO
  ax = ax or plt.gca()
  proj = dataset_reducido.values
  ax.plot(proj[:, 0], proj[:, 1], '.k')

  if images is not None:
      min_dist_2 = (0.001 * max(proj.max(0) - proj.min(0))) ** 2
      shown_images = np.array([2 * proj.max(0)])
      for i in range(proj.shape[0]):
          dist = np.sum((proj[i] - shown_images) ** 2, 1)
          if np.min(dist) < min_dist_2:
              # don't show points that are too close
              continue
          shown_images = np.vstack([shown_images, proj[i]])
          imagebox = offsetbox.AnnotationBbox(offsetbox.OffsetImage(images[i],2,cmap="gray"),proj[i])
          ax.add_artist(imagebox)

### Documentación de funciones

* `reducir_con_pca(data)`: recibe el dataset y retorna un DataFrame con 2 columnas obtenidas tras aplicar PCA

* `reducir_con_tsne(data, perplexity)`: recibe el dataset y un número positivo. Retorna un DataFrame con 2 columnas obtenidas tras aplicar TSNE. Cambiar el parámetro perplexity permite obtener diferentes resultados

* `reducir_con_mds(data, max_iter)`: recibe el dataset y un número positivo. Retorna un DataFrame con 2 columnas obtenidas tras aplicar MDS. Cambiar el parámetro max_iter permite obtener diferentes resultados.

* `reducir_con_umap(data, n_neighbors)`: recibe el dataset y un número positivo Retorna un DataFrame con 2 columnas obtenidas tras aplicar UMAP. Cambiar el parámetro `n_neighbors` permite obtener diferentes resultados.

* `visualizar(dataset_reducido, metodo)`. Recibe el dataset generado por las funciones detalladas anteriormente y un string para poner antes del dataset


Ejemplo de uso:

```
tsne_dataset_version_1 = reducir_con_tsne(dataset_surdoc, 30)
tsne_dataset_version_2 = reducir_con_tsne(dataset_surdoc, 60)
visualizar(tsne_dataset_version_1, "TSNE con perplexity de 30")

```

### Generar gráficos

In [0]:
# Gráfico 1 COMPLETAR



In [0]:
# Gráfico 2 COMPLETAR



### ¿Con ganas de ver más? 
En la siguiente página puedes ver un mapa interactivo con diferentes obras de arte posicionadas según reducción de dimensionalidad: http://niebla.ing.puc.cl/surdoc/

## Actividad 4

A continuación se presentará un dataset con información de flores y diversas funciones para aplicar diferentes métodos de reducción de dimensionalidad. Debe probar cada uno, visualizar el dataset resultando y finalmente indicar cual técnica permite mejor separar las clases de flores.

### Documentación de funciones (la misma que de la actividad 03)

* `reducir_con_pca(data)`: recibe el dataset y retorna un DataFrame con 2 columnas obtenidas tras aplicar PCA

* `reducir_con_tsne(data, perplexity)`: recibe el dataset y un número positivo. Retorna un DataFrame con 2 columnas obtenidas tras aplicar TSNE. Cambiar el parámetro perplexity permite obtener diferentes resultados

* `reducir_con_mds(data, max_iter)`: recibe el dataset y un número positivo. Retorna un DataFrame con 2 columnas obtenidas tras aplicar MDS. Cambiar el parámetro max_iter permite obtener diferentes resultados.

* `reducir_con_umap(data, n_neighbors)`: recibe el dataset y un número positivo Retorna un DataFrame con 2 columnas obtenidas tras aplicar UMAP. Cambiar el parámetro `n_neighbors` permite obtener diferentes resultados.


Ejemplo de uso:

```
lda_dataset = reducir_con_pca(dataset)

```

In [0]:
iris = datasets.load_iris()
dataset = iris.data
classes = iris.target

### Completar el siguiente código y luego responder la pregunta

In [0]:
# Vamos a usar como brush uno que funciona en mouseover.  NO TOCAR
# El parámetro fields indica que en cada gráfico seleccionará la marca que corresponda a la misma comuna.
brush = alt.selection_multi(on='mouseover', fields=["index"])


# Esta función creará algunos de los gráficos que vamos a usar.  NO TOCAR
def make_chart(data, titulo):
    """
    :param data: dataframe con los datos
    :return un Chart de Altair
    """
    source = pd.DataFrame(data.copy()).reset_index()
    source['classes'] = classes

    return  alt.Chart(source).mark_circle(size=100).encode(
        x='X',
        y='Y',
        color=alt.condition(brush, "classes:N", alt.value('lightgrey'),
                            legend=alt.Legend(title="Clases de flores")),
        # Esta configuración de opacidad nos permite ver mejor el elemento que estamos seleccionando.
        opacity= alt.condition(brush, alt.OpacityValue(1), alt.OpacityValue(0.1))
    ).add_selection(
        brush  # Este el brush es el mismo que definimos arriba.
    ).properties(
        width=250,
        height=250,
        title=titulo
    )


# Creamos los datos para cada uno de los métodos de reducción que tenemos. SI TOCAR
"""
COMPLETAR 
Crear los datasets mediante reducción de dimensionalidad, usar el ejemplo de la
casilla anterior
"""


# Se crean los gráficos para cada uno de los datos Usar la función make_chart. SI TOCAR
pca_chart = "COMPLETAR"
tsne_chart = "COMPLETAR"
mds_chart = "COMPLETAR"
umap_chart = "COMPLETAR"


# Los organizamos y mostramos. NO TOCAR
(((pca_chart & umap_chart)|( mds_chart & tsne_chart)).configure(tick=alt.TickConfig(thickness=3))  # Este configure nos permite hacer que las líneas del gráfico de LDA se configuren

### Respuesta

¿Dado las 5 técnicas, cual reduce mejor la información para separar cada clase? Explique como, dado un dato, reconoce que tipo de clase es con la reducción utilizada. 

